# AI Chess - Training ML Model

## Hướng dẫn sử dụng trên Google Colab

### Bước 1: Chuẩn bị
1. Upload file `chess_data.csv` (từ script `generate_data.py`) lên Colab
2. Chạy tất cả các cells từ trên xuống dưới
3. Download file `chess_model.h5` về máy

### Bước 2: Sau khi train
1. Đưa file `chess_model.h5` vào thư mục `data/` trong project
2. Chạy `evaluate.py` để kiểm tra hiệu suất

---

## 1. Cài đặt thư viện

In [ ]:
# Cài đặt python-chess nếu chưa có
!pip install python-chess -q

print("✓ Đã cài đặt các thư viện cần thiết")

## 2. Import thư viện

In [ ]:
import numpy as np
import pandas as pd
import chess
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

## 3. Upload dữ liệu

**Chú ý:** Bạn cần upload file `chess_data.csv` vào Colab trước khi chạy cell này.

Cách upload:
- Click vào icon folder ở sidebar bên trái
- Click nút Upload
- Chọn file `chess_data.csv`

In [ ]:
# Upload file từ máy tính
from google.colab import files
import io

print("Vui lòng chọn file 'chess_data.csv' để upload...")
uploaded = files.upload()

# Đọc dữ liệu
df = pd.read_csv('chess_data.csv')

print(f"\n✓ Đã upload thành công!")
print(f"Số lượng positions: {len(df)}")
print(f"\nMẫu dữ liệu đầu tiên:")
print(df.head())

# Thống kê
print(f"\nThống kê điểm số:")
print(df['score'].describe())

## 4. Xử lý dữ liệu

Chuyển FEN string thành tensor 3D (8x8x12)

In [ ]:
def fen_to_tensor(fen):
    """
    Chuyển FEN string thành tensor 3D (8x8x12)
    12 channels: 6 loại quân x 2 màu
    """
    board = chess.Board(fen)
    tensor = np.zeros((8, 8, 12), dtype=np.float32)
    
    # Mapping: piece_type -> channel index
    piece_to_channel = {
        chess.PAWN: 0,
        chess.KNIGHT: 1,
        chess.BISHOP: 2,
        chess.ROOK: 3,
        chess.QUEEN: 4,
        chess.KING: 5
    }
    
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            row = 7 - (square // 8)
            col = square % 8
            channel = piece_to_channel[piece.piece_type]
            
            # Thêm 6 nếu là quân đen
            if piece.color == chess.BLACK:
                channel += 6
            
            tensor[row, col, channel] = 1.0
    
    return tensor

# Test hàm
test_fen = df['fen'].iloc[0]
test_tensor = fen_to_tensor(test_fen)
print(f"Shape của tensor: {test_tensor.shape}")
print(f"Số lượng quân cờ: {int(test_tensor.sum())}")

## 5. Tạo dataset

In [ ]:
print("Đang chuyển đổi FEN thành tensors...")

# Chuyển tất cả FEN thành tensors
X = np.array([fen_to_tensor(fen) for fen in df['fen']])
y = df['score'].values.astype(np.float32)

print(f"\nShape của X: {X.shape}")
print(f"Shape của y: {y.shape}")

# Normalize điểm số (để training tốt hơn)
y_mean = y.mean()
y_std = y.std()
y_normalized = (y - y_mean) / y_std

print(f"\nĐiểm số trước normalize: mean={y_mean:.2f}, std={y_std:.2f}")
print(f"Điểm số sau normalize: mean={y_normalized.mean():.4f}, std={y_normalized.std():.4f}")

## 6. Chia train/validation

In [ ]:
# Chia 80% train, 20% validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y_normalized, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")

## 7. Xây dựng Model

Sử dụng CNN (Convolutional Neural Network) vì phù hợp với dữ liệu dạng bàn cờ 2D

In [ ]:
def create_model():
    """
    Tạo CNN model để đánh giá bàn cờ
    """
    model = keras.Sequential([
        # Input layer
        layers.Input(shape=(8, 8, 12)),
        
        # Convolutional layers
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        
        # Flatten
        layers.Flatten(),
        
        # Dense layers
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        
        # Output layer (dự đoán 1 số - điểm số)
        layers.Dense(1)
    ])
    
    return model

# Tạo model
model = create_model()

# In kiến trúc
model.summary()

## 8. Compile model

In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',  # Mean Squared Error
    metrics=['mae']  # Mean Absolute Error
)

print("✓ Đã compile model")

## 9. Training

**Chú ý:** Training có thể mất 10-30 phút tùy thuộc vào:
- Số lượng dữ liệu
- Số epochs
- Hardware (GPU/CPU)

Trên Google Colab, hãy bật GPU để nhanh hơn:
- Runtime → Change runtime type → Hardware accelerator → GPU

In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=0.00001
    )
]

# Training
print("Bắt đầu training...\n")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Hoàn thành training!")

## 10. Visualize kết quả training

In [ ]:
# Plot loss
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 11. Đánh giá model

In [ ]:
# Đánh giá trên validation set
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)

print(f"Validation Loss (MSE): {val_loss:.4f}")
print(f"Validation MAE: {val_mae:.4f}")

# Dự đoán một vài samples
print("\nMột vài dự đoán:")
print("-" * 60)

for i in range(5):
    sample = X_val[i:i+1]
    true_score = y_val[i]
    pred_score = model.predict(sample, verbose=0)[0][0]
    
    # Denormalize
    true_score_denorm = true_score * y_std + y_mean
    pred_score_denorm = pred_score * y_std + y_mean
    
    print(f"Sample {i+1}: True={true_score_denorm:.1f}, Pred={pred_score_denorm:.1f}, Diff={abs(true_score_denorm-pred_score_denorm):.1f}")

## 12. Lưu model

**Quan trọng:** Sau khi chạy cell này, download file `chess_model.h5` về máy!

In [ ]:
# Lưu model
model.save('chess_model.h5')

print("✓ Đã lưu model vào 'chess_model.h5'")
print("\n" + "="*60)
print("HOÀN TẤT!")
print("="*60)
print("\nBước tiếp theo:")
print("1. Download file 'chess_model.h5' về máy")
print("2. Đặt file vào thư mục 'data/' trong project")
print("3. Chạy 'evaluate.py' để kiểm tra hiệu suất")
print("="*60)

## 13. (Optional) Test model với FEN tùy chỉnh

In [ ]:
# Test với vị trí bắt đầu
test_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"

test_tensor = fen_to_tensor(test_fen)
test_batch = np.expand_dims(test_tensor, axis=0)

pred_normalized = model.predict(test_batch, verbose=0)[0][0]
pred_score = pred_normalized * y_std + y_mean

print(f"FEN: {test_fen}")
print(f"Predicted score: {pred_score:.2f}")
print("\n(Điểm số nên gần 0 vì vị trí bắt đầu là cân bằng)")

---

# Hướng dẫn cải thiện Model

## Nếu model chưa đạt yêu cầu (thắng Random < 60%):

### 1. Tăng dữ liệu training
- Chạy `generate_data.py` với nhiều ván hơn (500-1000 ván)
- Càng nhiều dữ liệu càng tốt

### 2. Cải thiện kiến trúc model
- Thêm nhiều Conv2D layers hơn
- Tăng số filters (32 → 64 → 128 → 256)
- Thử ResNet hoặc các kiến trúc phức tạp hơn

### 3. Tăng epochs
- Training lâu hơn (50-100 epochs)
- Sử dụng EarlyStopping để tránh overfit

### 4. Data augmentation
- Lật bàn cờ theo chiều ngang (mirror)
- Xoay bàn cờ
- Tăng gấp đôi/gấp ba dữ liệu

### 5. Hyperparameter tuning
- Thử các learning rate khác nhau (0.0001, 0.001, 0.01)
- Thử batch size khác (32, 64, 128)
- Thử optimizer khác (Adam, SGD, RMSprop)

---